In [1]:
# Multilayer perceptron (Keras)

In [1]:
import pandas as pd
import numpy as np
from statsmodels.robust.scale import mad
import os
from keras.models import Sequential
from keras.layers import Dense

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def mergeData():
    SaveFile_Name = 'data.csv'
    file_list = os.listdir('data')
    df = pd.read_csv('data'+'\\'+file_list[0])
    df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False)
    for i in range(1,len(file_list)):
        df = pd.read_csv('data'+'\\'+file_list[i])
        df.to_csv(SaveFile_Name,encoding="utf_8_sig",index=False, header=False, mode='a+')

In [3]:
def readData():
  train = pd.read_csv("data.csv")
  return train

In [4]:
def changeYear(data):
    for i in range(0,data.shape[0]):
        Date=data["日期"][i].split('/')
        year,month,date=Date[0],Date[1],Date[2]
        year=str(int(year)+1911)
        data.loc[i,"日期"]=year+'/'+month+'/'+date
    return data

In [5]:
def augFeatures(data):
  data["日期"] = pd.to_datetime(data["日期"])
  data["年"] = data["日期"].dt.year
  data["月"] = data["日期"].dt.month
  data["日"] = data["日期"].dt.day
  data["第幾日"] = data["日期"].dt.dayofweek
  return data

In [6]:
def normalizeFunction(x):
    return ((x - np.mean(x)) / (np.max(x) - np.min(x)))

In [7]:
def normalize(data):
    for i in range(0,data.shape[0]):
        if data["漲跌價差"][i]=='X0.00':
            data.loc[i,"漲跌價差"]=str(int(data["收盤價"][i])-int(data["收盤價"][i-1]))
    data=data.drop(["日期"], axis=1)
    data=data.drop(["成交股數"], axis=1)
    data=data.drop(["成交金額"], axis=1)
    data=data.drop(["漲跌價差"], axis=1)
    data=data.drop(["成交筆數"], axis=1)
    data=data.convert_objects(convert_numeric=True)
    datanormalize=data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return datanormalize

In [8]:
def predictAns(data):
    for i in range(0,data.shape[0]-1):
        data.loc[i,"結果"]=data["開盤價"][i+1]
    data=data.drop([data.shape[0]-1])
    return data

In [12]:
mergeData()
getdata=readData()
changedata=changeYear(getdata)
adddata=augFeatures(changedata)
addans=predictAns(adddata)
train_x=addans
train_x=train_x.drop(["結果"], axis=1)
train_x=normalize(train_x)
train_x=train_x.as_matrix()
train_y=addans["結果"]
train_y=train_y.as_matrix()

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  # Remove the CWD from sys.path while we load stuff.
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [15]:
model=Sequential()
model.add(Dense(units=256,input_dim=train_x.shape[1],kernel_initializer='normal',activation='relu'))
model.add(Dense(units=1,kernel_initializer='normal',activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               2304      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 257       
Total params: 2,561
Trainable params: 2,561
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
train_history=model.fit(x=train_x,y=train_y,validation_split=0.2,epochs=10,batch_size=200,verbose=2)

Train on 4952 samples, validate on 1238 samples
Epoch 1/10


InvalidArgumentError: Received a label value of 209 which is outside the valid range of [0, 1).  Label values: 57 91 63 44 58 73 76 64 176 52 68 62 47 100 54 111 48 85 69 97 91 145 100 155 56 96 78 60 63 85 77 142 90 189 103 139 159 66 84 61 61 103 63 59 45 47 52 122 79 68 65 90 68 58 132 127 51 145 35 111 61 65 108 60 159 90 65 101 68 181 59 55 81 123 59 67 140 69 38 67 72 87 43 75 62 110 89 63 62 62 63 54 67 163 98 64 75 57 86 49 193 52 76 62 102 109 66 61 83 62 166 56 65 107 148 78 86 155 62 142 62 52 65 61 87 78 48 85 107 159 100 142 65 198 140 80 52 76 130 47 106 58 63 55 136 86 99 188 70 86 55 142 59 83 106 209 73 82 60 51 93 147 57 50 59 76 74 42 74 97 155 164 47 60 113 99 100 55 58 62 83 63 65 68 90 79 76 54 133 175 85 46 99 56 58 72 162 160 46 207
	 [[{{node loss_2/dense_4_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}} = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _class=["loc:@train...s_grad/mul"], _device="/job:localhost/